In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-sparse  -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-geometric

!pip install --use-deprecated=legacy-resolver karateclub networkx numpy pandas matplotlib scikit-learn

!pip install torch torchvision torchaudio
!pip install torch-geometric \
    -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html


!pip install optuna
!pip install karateclub

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 66.5 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 125.5 MB/s eta 0:00:00
  Created wheel for karateclub: filename=karateclub-1.3.3-py3-none-any.whl size=101979 sha256=ff1bf582ff973e2

In [2]:
# Imports
import joblib
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d
from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from sklearn.metrics import accuracy_score, f1_score
from torch_geometric.transforms import OneHotDegree
import optuna
import pandas as pd
import time, os, psutil
from sklearn.metrics import roc_auc_score
import numpy as np
from karateclub import NetLSD, Graph2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from torch_geometric.utils import to_networkx
import networkx as nx
import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cuda.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_sparse/_version_cuda.so
  import torch_geometric.typing


In [3]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/InformationSystems/Classification"
RESULTS_DIR = f"{BASE_DIR}/results"
MODELS_DIR = f"{BASE_DIR}/models"
EMBEDDINGS_DIR = f"{BASE_DIR}/embeddings"
CLASSIF_RESULTS_DIR = f"{BASE_DIR}/results/classification"

os.makedirs(EMBEDDINGS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(CLASSIF_RESULTS_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Mounted at /content/drive


In [4]:
def sanitize_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    Replace NaN/Inf values in embeddings and ensure a clean float32 array.
    This is useful for karateclub embeddings that may occasionally produce
    unstable values on some graphs.
    """
    emb = np.asarray(embeddings, dtype=np.float32)
    # Replace NaN and +/- Inf with 0.0
    emb = np.nan_to_num(emb, nan=0.0, posinf=0.0, neginf=0.0)
    return emb

In [5]:
def filter_enzymes_graphs(graphs, labels, min_nodes: int = 3):
    """
    Special handling for ENZYMES: remove very small graphs that can cause
    numerical issues for NetLSD / Graph2Vec.
    Returns filtered (graphs, labels) and prints how many were removed.
    """
    if len(graphs) == 0:
        return graphs, labels

    mask = [g.number_of_nodes() >= min_nodes for g in graphs]
    if not any(mask):
        print("WARNING: All ENZYMES graphs would be filtered out. Skipping filtering.")
        return graphs, labels

    filtered_graphs = [g for g, keep in zip(graphs, mask) if keep]
    if isinstance(labels, np.ndarray):
       filtered_labels = labels[np.array(mask)]
    else:
        filtered_labels = [y for y, keep in zip(labels, mask) if keep]

    removed = len(graphs) - len(filtered_graphs)
    print(f"ENZYMES filtering: removed {removed} graphs with < {min_nodes} nodes, kept {len(filtered_graphs)} graphs.")
    return filtered_graphs, filtered_labels

In [6]:
# GIN Model Definition

class GIN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes, num_layers=5, dropout=0.5):
        super(GIN, self).__init__()
        layers = []
        in_dim = num_features
        for _ in range(num_layers):
            nn = Sequential(Linear(in_dim, hidden_dim), ReLU(), Linear(hidden_dim, hidden_dim))
            layers.append(GINConv(nn))
            in_dim = hidden_dim
        self.convs = torch.nn.ModuleList(layers)
        self.bns = torch.nn.ModuleList([BatchNorm1d(hidden_dim) for _ in range(num_layers)])
        self.fc1 = Linear(hidden_dim, hidden_dim)
        self.fc2 = Linear(hidden_dim, num_classes)
        self.dropout = dropout

    def forward(self, x, edge_index, batch):
        for conv, bn in zip(self.convs, self.bns):
            x = F.relu(conv(x, edge_index))
            x = bn(x)
        x = global_add_pool(x, batch)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.fc2(x)
        return x

In [7]:
# Training

def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(DEVICE)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation

def evaluate(model, loader, criterion):
    model.eval()
    preds, labels, probs = [], [], []
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for data in loader:
            data = data.to(DEVICE)
            out = model(data.x, data.edge_index, data.batch)

            loss = criterion(out, data.y)
            total_loss += loss.item()
            num_batches += 1

            pred = out.argmax(dim=1)
            preds.extend(pred.cpu().numpy())
            labels.extend(data.y.cpu().numpy())
            probs.extend(F.softmax(out, dim=1).cpu().numpy())  # probabilities for AUC

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    avg_loss = total_loss / max(1, num_batches)

    try:
        if len(np.unique(labels)) == 2:
            auc = roc_auc_score(labels, np.array(probs)[:, 1])
        else:
            auc = roc_auc_score(labels, probs, multi_class='ovr')
    except ValueError:
        auc = np.nan  # if there are not enough samples for AUC

    return acc, f1, auc, avg_loss

In [8]:
def get_gin_embeddings(model, loader):
    """Return graph-level embeddings (after global_add_pool) and labels."""
    model.eval()
    all_emb = []
    all_labels = []
    with torch.no_grad():
        for data in loader:
            data = data.to(DEVICE)
            x, edge_index, batch = data.x, data.edge_index, data.batch
            # forward μέχρι το pooling
            for conv, bn in zip(model.convs, model.bns):
                x = F.relu(conv(x, edge_index))
                x = bn(x)
            x = global_add_pool(x, batch)
            all_emb.append(x.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
    embeddings = np.concatenate(all_emb, axis=0)
    labels = np.array(all_labels)
    return embeddings, labels

In [9]:
def run_gin_pipeline(
    dataset_name,
    seed,
    use_optuna,
    w_acc,
    w_f1,
    w_auc,
    hidden_dim,
    epochs,
    batch_size=32,
    n_trials=15,
):

    # Experiment ID (used in logs and embeddings path)
    experiment_id = str(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    # Load dataset
    dataset = TUDataset(root='data/TUDataset', name=dataset_name).shuffle()

    # If no node features use one-hot degree features
    if dataset.num_features == 0 or dataset[0].x is None:
        print("Dataset has no node features. Applying OneHotDegree transform...")

        # Find maximum degree across all graphs
        max_degree = 0
        for data in dataset:
            deg = torch.bincount(data.edge_index[0], minlength=data.num_nodes)
            max_degree = max(max_degree, int(deg.max()))

        # Apply transform
        oh_transform = OneHotDegree(max_degree=max_degree)
        dataset = TUDataset(
            root='data/TUDataset',
            name=dataset_name,
            transform=oh_transform
        ).shuffle()

        num_node_features = max_degree + 1
    else:
        num_node_features = dataset.num_features

    # Train/test split
    train_dataset = dataset[:int(0.8 * len(dataset))]
    test_dataset = dataset[int(0.8 * len(dataset)):]
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    print(f"Loaded dataset {dataset_name}: {len(dataset)} graphs, {num_node_features} node features, {dataset.num_classes} classes")

    def objective(trial):
        num_layers = trial.suggest_int("num_layers", 3, 6)
        dropout = trial.suggest_float("dropout", 0.0, 0.6)
        lr = trial.suggest_loguniform("lr", 1e-4, 1e-2)
        weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)

        model = GIN(num_node_features, hidden_dim, dataset.num_classes, num_layers, dropout).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        criterion = torch.nn.CrossEntropyLoss()

        for epoch in range(5):  # fewer epochs for fast tuning
            train(model, train_loader, optimizer, criterion)

        acc, f1, auc, _ = evaluate(model, test_loader, criterion)
        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    start_generation = time.time()
    if use_optuna:
        print("Running Optuna for hyperparameter tuning...")
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print(f"Best hyperparameters: {best_params}")
    else:
      best_params = { "num_layers": 5, "dropout": 0.5, "lr": 0.001, "weight_decay": 1e-4}
      print(f"Using default hyperparameters: {best_params}")

    generation_time = time.time() - start_generation

    # Final Training with best parameters

    print("\nRunning final training GIN...")
    print(best_params)

    model = GIN(num_node_features, hidden_dim, dataset.num_classes,
                num_layers=best_params["num_layers"], dropout=best_params["dropout"]).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
    criterion = torch.nn.CrossEntropyLoss()

    history = []
    start_time = time.time()
    eval_acc, eval_f1, eval_auc, eval_loss, eval_epoch = 0, 0, 0, 1e9, 0
    best_loss_for_best_epoch = 1e9
    for epoch in range(1, epochs + 1):
        loss = train(model, train_loader, optimizer, criterion)
        acc, f1, auc, e_loss = evaluate(model, test_loader, criterion)
        if acc > eval_acc:
          #edo mipos to allakso na einai kai edo sindiasmos me weights poy eixe kai sto optuna
          eval_acc, eval_f1, eval_auc, eval_loss, eval_epoch = acc, f1, auc, e_loss, epoch
          best_loss_for_best_epoch = e_loss

        elapsed = time.time() - start_time
        print(f"Epoch {epoch:03d} | Loss={loss:.4f} | TestAcc={acc:.3f} | F1={f1:.3f} | AUC={auc:.3f} | Time={elapsed:.2f}s")
        history.append([epoch, loss, acc, f1, auc, elapsed])

    training_time = time.time() - start_time
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB


    # Save GIN embeddings for the whole dataset

    full_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    gin_embeddings, gin_labels = get_gin_embeddings(model, full_loader)

    gin_exp_dir = os.path.join(EMBEDDINGS_DIR, "GIN", dataset_name, experiment_id)
    os.makedirs(gin_exp_dir, exist_ok=True)
    np.save(os.path.join(gin_exp_dir, "embeddings.npy"), gin_embeddings)
    np.save(os.path.join(gin_exp_dir, "labels.npy"), gin_labels)

    # Log file save

    summary_path = f"{CLASSIF_RESULTS_DIR}/gin.csv"
    os.makedirs("results", exist_ok=True)


    summary_data = {
        "method": "GIN",
        "seed": seed,
        "dataset": dataset_name,
        "optimization_enabled": "yes" if use_optuna else "no",
        "embedding_dimension": hidden_dim,
        "objective_weights": f"({w_acc},{w_f1},{w_auc})",
        "num_layers": best_params["num_layers"],
        "dropout": best_params["dropout"],
        "lr": best_params["lr"],
        "weight_decay": best_params["weight_decay"],
        "epochs": epochs,
        "best_epoch": eval_epoch,
        "best_loss": round(float(best_loss_for_best_epoch), 4),
        "eval_loss": round(float(eval_loss), 4),
        "eval_acc": round(eval_acc, 4),
        "eval_f1": round(eval_f1, 4),
        "eval_auc": round(eval_auc, 4),
        "training_time (s)": round(training_time, 2),
        "generation_time (s)": round(generation_time, 2),
        "memory_usage (MB)": round(memory_usage, 2)
    }

    df = pd.DataFrame([summary_data])

    # Append mode (keep all trainings)
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"\nTraining summary stored in : {summary_path}")
    print(df)

    # Save best model (weights + metadata)
    gin_ckpt = {
        "state_dict": model.state_dict(),
        "num_node_features": num_node_features,
        "hidden_dim": hidden_dim,
        "num_classes": dataset.num_classes,
        "num_layers": best_params["num_layers"],
        "dropout": best_params["dropout"],
    }
    torch.save(gin_ckpt, f"{MODELS_DIR}/GIN_{dataset_name}_{experiment_id}.pth")
    print(f"Saved model: {MODELS_DIR}/GIN_{dataset_name}_{experiment_id}.pth")


In [10]:
def run_graph2vec_pipeline(
    dataset_name,
    seed,
    w_acc=0.5,
    w_f1=0.3,
    w_auc=0.2,
    embedding_dim=128,
    epochs=50,
    test_size=0.2,
    use_optuna=True,
    n_trials=20,
):
    """
    Pipeline for graph classification using Graph2Vec embeddings + SVM,
    with optional Optuna-based hyperparameter tuning and special handling
    for ENZYMES + embedding sanitization.
    """
    # Experiment ID
    experiment_id = str(seed)
    np.random.seed(seed)

    # Load dataset
    dataset = TUDataset(root='data/TUDataset', name=dataset_name).shuffle()
    print(f"Loaded dataset {dataset_name} for Graph2Vec: {len(dataset)} graphs, {dataset.num_classes} classes")

    # Convert PyG graphs to NetworkX graphs
    graphs = []
    labels = []
    for data in dataset:
        g = to_networkx(data, to_undirected=True)
        graphs.append(g)
        labels.append(int(data.y.item()))

    labels = np.array(labels)

    # Special handling for ENZYMES (filter very small graphs)
    if dataset_name.upper() == "ENZYMES":
        graphs, labels = filter_enzymes_graphs(graphs, labels, min_nodes=3)

    # Outer train/test split on graphs
    train_graphs, test_graphs, y_train, y_test = train_test_split(
        graphs,
        labels,
        test_size=test_size,
        random_state=seed,
        stratify=labels,
    )

    opt_time = 0.0

    def objective(trial):
        # Hyperparameters for the SVM classifier
        C = trial.suggest_loguniform("C", 1e-2, 1e2)
        gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)

        # Inner train/validation split on graphs
        inner_tr_graphs, inner_val_graphs, y_tr, y_val = train_test_split(
            train_graphs,
            y_train,
            test_size=0.2,
            random_state=seed,
            stratify=y_train,
        )

        all_graphs = inner_tr_graphs + inner_val_graphs

        # Fit Graph2Vec on all (transductive setting) and slice embeddings
        g2v = Graph2Vec(dimensions=embedding_dim, wl_iterations=2, epochs=epochs, workers=os.cpu_count())
        g2v.fit(all_graphs)
        emb_all = sanitize_embeddings(g2v.get_embedding())

        X_tr = emb_all[:len(inner_tr_graphs)]
        X_val = emb_all[len(inner_tr_graphs):]

        clf = SVC(kernel="rbf", probability=True, C=C, gamma=gamma, random_state=seed)
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        y_prob = clf.predict_proba(X_val)

        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average="weighted")
        try:
            if len(np.unique(y_val)) == 2:
                auc = roc_auc_score(y_val, y_prob[:, 1])
            else:
                auc = roc_auc_score(y_val, y_prob, multi_class="ovr")
        except ValueError:
            auc = np.nan

        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    if use_optuna:
        print("Running Optuna for Graph2Vec+SVM hyperparameter tuning...")
        start_opt = time.time()
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        opt_time = time.time() - start_opt
        print(f"Best hyperparameters (Graph2Vec+SVM): {best_params}")
    else:
        best_params = {"C": 1.0, "gamma": "scale"}
        print(f"Using default SVM hyperparameters: {best_params}")

    # Final embedding + training using best hyperparameters
    print("Running final Graph2Vec embedding on train+test graphs...")
    all_graphs_final = train_graphs + test_graphs
    start_embed = time.time()
    g2v = Graph2Vec(dimensions=embedding_dim, wl_iterations=2, epochs=epochs, workers=os.cpu_count())
    g2v.fit(all_graphs_final)
    emb_all = sanitize_embeddings(g2v.get_embedding())
    embed_time = time.time() - start_embed

    X_train = emb_all[:len(train_graphs)]
    X_test = emb_all[len(train_graphs):]

    print("Training final SVM on Graph2Vec embeddings...")
    start_train = time.time()
    clf = SVC(kernel="rbf", probability=True, C=best_params["C"], gamma=best_params["gamma"], random_state=seed)
    clf.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Evaluation on held-out test graphs
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    try:
        if len(np.unique(y_test)) == 2:
            auc = roc_auc_score(y_test, y_prob[:, 1])
        else:
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr")
    except ValueError:
        auc = np.nan

    score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))


    # Save Graph2Vec embeddings (for all graphs) + labels

    g2v_exp_dir = os.path.join(EMBEDDINGS_DIR, "Graph2Vec", dataset_name, experiment_id)
    os.makedirs(g2v_exp_dir, exist_ok=True)
    np.save(os.path.join(g2v_exp_dir, "embeddings.npy"), emb_all)
    np.save(os.path.join(g2v_exp_dir, "labels.npy"), np.array(labels))

    # Save fitted Graph2Vec model
    g2v_model_path = f"{MODELS_DIR}/Graph2Vec_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(g2v, g2v_model_path)

    # Save trained SVM classifier
    svm_model_path = f"{MODELS_DIR}/Graph2Vec_SVM_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(clf, svm_model_path)


    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB

    print(f"Graph2Vec Results on {dataset_name} -> Acc: {acc:.3f}, F1: {f1:.3f}, AUC: {auc:.3f}, Score: {score:.3f}")
    print(f"Embedding time: {embed_time:.2f}s | SVM training time: {train_time:.2f}s | Optuna time: {opt_time:.2f}s | Memory usage: {memory_usage:.2f} MB")

    # Log summary to CSV
    summary_path = f"{CLASSIF_RESULTS_DIR}/g2v.csv"

    summary_data = {
        "method": "Graph2Vec",
        "seed": seed,
        "dataset": dataset_name,
        "embedding_dimension": embedding_dim,
        "optuna_enabled": "yes" if use_optuna else "no",
        "C": best_params["C"],
        "gamma": best_params["gamma"],
        "acc": round(float(acc), 4),
        "f1": round(float(f1), 4),
        "auc": round(float(auc) if not np.isnan(auc) else -1, 4),
        "score": round(float(score), 4),
        "embedding_time (s)": round(embed_time, 2),
        "svm_training_time (s)": round(train_time, 2),
        "optuna_time (s)": round(opt_time, 2),
        "memory_usage (MB)": round(memory_usage, 2),
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"Graph2Vec summary stored in: {summary_path}")


In [11]:
def run_netlsd_pipeline(
    dataset_name,
    seed,
    w_acc=0.5,
    w_f1=0.3,
    w_auc=0.2,
    test_size=0.2,
    use_optuna=True,
    n_trials=20,
):
    """
    Pipeline for graph classification using NetLSD embeddings + SVM,
    with optional Optuna-based hyperparameter tuning and ENZYMES filtering.
    """
    # Experiment ID
    experiment_id = str(seed)
    np.random.seed(seed)
    # Load dataset
    dataset = TUDataset(root='data/TUDataset', name=dataset_name).shuffle()
    print(f"Loaded dataset {dataset_name} for NetLSD: {len(dataset)} graphs, {dataset.num_classes} classes")

    # Convert PyG graphs to NetworkX graphs
    graphs = []
    labels = []
    for data in dataset:
        g = to_networkx(data, to_undirected=True)
        graphs.append(g)
        labels.append(int(data.y.item()))

    labels = np.array(labels)

    # Special handling for ENZYMES
    if dataset_name.upper() == "ENZYMES":
        graphs, labels = filter_enzymes_graphs(graphs, labels, min_nodes=3)

    # Outer train/test split on graphs
    train_graphs, test_graphs, y_train, y_test = train_test_split(
        graphs,
        labels,
        test_size=test_size,
        random_state=seed,
        stratify=labels,
    )

    opt_time = 0.0

    def objective(trial):
        # Hyperparameters for the SVM classifier
        C = trial.suggest_loguniform("C", 1e-2, 1e2)
        gamma = trial.suggest_loguniform("gamma", 1e-4, 1e1)

        # Inner train/validation split on graphs
        inner_tr_graphs, inner_val_graphs, y_tr, y_val = train_test_split(
            train_graphs,
            y_train,
            test_size=0.2,
            random_state=seed,
            stratify=y_train,
        )

        all_graphs = inner_tr_graphs + inner_val_graphs

        # Fit NetLSD on all and slice embeddings
        netlsd = NetLSD()
        netlsd.fit(all_graphs)
        emb_all = sanitize_embeddings(netlsd.get_embedding())

        X_tr = emb_all[:len(inner_tr_graphs)]
        X_val = emb_all[len(inner_tr_graphs):]

        clf = SVC(kernel="rbf", probability=True, C=C, gamma=gamma, random_state=seed)
        clf.fit(X_tr, y_tr)
        y_pred = clf.predict(X_val)
        y_prob = clf.predict_proba(X_val)

        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average="weighted")
        try:
            if len(np.unique(y_val)) == 2:
                auc = roc_auc_score(y_val, y_prob[:, 1])
            else:
                auc = roc_auc_score(y_val, y_prob, multi_class="ovr")
        except ValueError:
            auc = np.nan

        score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))
        return score

    if use_optuna:
        print("Running Optuna for NetLSD+SVM hyperparameter tuning...")
        start_opt = time.time()
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        opt_time = time.time() - start_opt
        print(f"Best hyperparameters (NetLSD+SVM): {best_params}")
    else:
        best_params = {"C": 1.0, "gamma": "scale"}
        print(f"Using default SVM hyperparameters: {best_params}")

    # Final embedding + training using best hyperparameters
    print("Running final NetLSD embedding on train+test graphs...")
    all_graphs_final = train_graphs + test_graphs
    start_embed = time.time()
    netlsd = NetLSD()
    netlsd.fit(all_graphs_final)
    emb_all = sanitize_embeddings(netlsd.get_embedding())
    embed_time = time.time() - start_embed

    X_train = emb_all[:len(train_graphs)]
    X_test = emb_all[len(train_graphs):]

    print("Training final SVM on NetLSD embeddings...")
    start_train = time.time()
    clf = SVC(kernel="rbf", probability=True, C=best_params["C"], gamma=best_params["gamma"], random_state=seed)
    clf.fit(X_train, y_train)
    train_time = time.time() - start_train

    # Evaluation on held-out test graphs
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    try:
        if len(np.unique(y_test)) == 2:
            auc = roc_auc_score(y_test, y_prob[:, 1])
        else:
            auc = roc_auc_score(y_test, y_prob, multi_class="ovr")
    except ValueError:
        auc = np.nan

    score = (w_acc * acc) + (w_f1 * f1) + (w_auc * (0 if np.isnan(auc) else auc))


    # Save NetLSD embeddings (all graphs) + labels

    netlsd_exp_dir = os.path.join(EMBEDDINGS_DIR, "NetLSD",dataset_name, experiment_id)
    os.makedirs(netlsd_exp_dir, exist_ok=True)
    np.save(os.path.join(netlsd_exp_dir, "embeddings.npy"), emb_all)
    np.save(os.path.join(netlsd_exp_dir, "labels.npy"), np.array(labels))

    # Save fitted NetLSD model
    netlsd_model_path = f"{MODELS_DIR}/NetLSD_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(netlsd, netlsd_model_path)

    # Save trained SVM classifier
    svm_model_path = f"{MODELS_DIR}/NetLSD_SVM_{dataset_name}_{experiment_id}.joblib"
    joblib.dump(clf, svm_model_path)


    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 ** 2)  # in MB

    print(f"NetLSD Results on {dataset_name} -> Acc: {acc:.3f}, F1: {f1:.3f}, AUC: {auc:.3f}, Score: {score:.3f}")
    print(f"Embedding time: {embed_time:.2f}s | SVM training time: {train_time:.2f}s | Optuna time: {opt_time:.2f}s | Memory usage: {memory_usage:.2f} MB")

    # Log summary to CSV
    summary_path = f"{CLASSIF_RESULTS_DIR}/netlsd.csv"

    summary_data = {
        "method": "NetLSD",
        "seed": seed,
        "optuna_enabled": "yes" if use_optuna else "no",
        "C": best_params["C"],
        "gamma": best_params["gamma"],
        "acc": round(float(acc), 4),
        "f1": round(float(f1), 4),
        "auc": round(float(auc) if not np.isnan(auc) else -1, 4),
        "score": round(float(score), 4),
        "embedding_time (s)": round(embed_time, 2),
        "svm_training_time (s)": round(train_time, 2),
        "optuna_time (s)": round(opt_time, 2),
        "memory_usage (MB)": round(memory_usage, 2),
    }

    df = pd.DataFrame([summary_data])
    if os.path.exists(summary_path):
        df.to_csv(summary_path, mode='a', index=False, header=False)
    else:
        df.to_csv(summary_path, index=False)

    print(f"NetLSD summary stored in: {summary_path}")


In [12]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_gin_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          use_optuna=True,
          w_acc=0.5,
          w_f1=0.3,
          w_auc=0.2,
          hidden_dim=64,
          epochs=200,
          batch_size=32,
          n_trials=10,
      )

Processing...
Done!


Dataset has no node features. Applying OneHotDegree transform...


[I 2026-01-22 11:05:31,897] A new study created in memory with name: no-name-867cc728-6507-4353-b4f6-cd54306609a9


Loaded dataset IMDB-MULTI: 1500 graphs, 89 node features, 3 classes
Running Optuna for hyperparameter tuning...


[I 2026-01-22 11:05:35,388] Trial 0 finished with value: 0.3808476513749058 and parameters: {'num_layers': 6, 'dropout': 0.33434670268252814, 'lr': 0.0013089563985149541, 'weight_decay': 0.0007165338468342581}. Best is trial 0 with value: 0.3808476513749058.
[I 2026-01-22 11:05:37,669] Trial 1 finished with value: 0.459772480997108 and parameters: {'num_layers': 3, 'dropout': 0.11505963633519778, 'lr': 0.00046598131355125703, 'weight_decay': 0.00020963436185342127}. Best is trial 1 with value: 0.459772480997108.
[I 2026-01-22 11:05:39,718] Trial 2 finished with value: 0.3821035737484786 and parameters: {'num_layers': 4, 'dropout': 0.2245986286091399, 'lr': 0.000114634640762874, 'weight_decay': 0.00010749554972788053}. Best is trial 1 with value: 0.459772480997108.
[I 2026-01-22 11:05:41,682] Trial 3 finished with value: 0.46198086202559363 and parameters: {'num_layers': 4, 'dropout': 0.0988857215282207, 'lr': 0.00042478568601898034, 'weight_decay': 3.2466981984519287e-06}. Best is tria

...

Best hyperparameters: {'num_layers': 3, 'dropout': 0.5025752272451087, 'lr': 0.0004369195157162352, 'weight_decay': 0.0009578053481133659}

Running final training GIN...
{'num_layers': 3, 'dropout': 0.5025752272451087, 'lr': 0.0004369195157162352, 'weight_decay': 0.0009578053481133659}
Epoch 001 | Loss=1.1145 | TestAcc=0.316 | F1=0.152 | AUC=0.144 | Time=0.04s
Epoch 002 | Loss=0.7772 | TestAcc=0.316 | F1=0.152 | AUC=0.042 | Time=0.08s
Epoch 003 | Loss=0.6676 | TestAcc=0.263 | F1=0.169 | AUC=0.112 | Time=0.12s
Epoch 004 | Loss=0.6047 | TestAcc=0.316 | F1=0.281 | AUC=0.186 | Time=0.16s
Epoch 005 | Loss=0.5983 | TestAcc=0.263 | F1=0.263 | AUC=0.196 | Time=0.20s
Epoch 006 | Loss=0.4435 | TestAcc=0.342 | F1=0.357 | AUC=0.285 | Time=0.24s
Epoch 007 | Loss=0.3582 | TestAcc=0.447 | F1=0.467 | AUC=0.452 | Time=0.28s
Epoch 008 | Loss=0.4419 | TestAcc=0.605 | F1=0.609 | AUC=0.529 | Time=0.33s
Epoch 009 | Loss=0.4919 | TestAcc=0.684 | F1=0.664 | AUC=0.702 | Time=0.37s
Epoch 010 | Loss=0.3090 | Tes

[I 2026-01-22 11:13:17,068] A new study created in memory with name: no-name-f29e21b1-9dee-4ddf-a2b9-2de388592fc8



Training summary stored in : /content/drive/MyDrive/InformationSystems/Classification/results/classification/gin.csv
  method  seed dataset optimization_enabled  embedding_dimension  \
0    GIN    43   MUTAG                  yes                   64   

  objective_weights  num_layers   dropout        lr  weight_decay  epochs  \
0     (0.5,0.3,0.2)           3  0.502575  0.000437      0.000958     200   

   best_epoch  best_loss  eval_loss  eval_acc  eval_f1  eval_auc  \
0          48     0.2023     0.2023    0.9737   0.9734    0.9936   

   training_time (s)  generation_time (s)  memory_usage (MB)  
0               9.02                 2.18            1461.79  
Saved model: /content/drive/MyDrive/InformationSystems/Classification/models/GIN_MUTAG_43.pth
Loaded dataset MUTAG: 188 graphs, 7 node features, 2 classes
Running Optuna for hyperparameter tuning...


[I 2026-01-22 11:13:17,347] Trial 0 finished with value: 0.5576063722524702 and parameters: {'num_layers': 6, 'dropout': 0.4153601045616967, 'lr': 0.0004660340572952553, 'weight_decay': 2.892937038967969e-06}. Best is trial 0 with value: 0.5576063722524702.
[I 2026-01-22 11:13:17,571] Trial 1 finished with value: 0.8680975307291097 and parameters: {'num_layers': 5, 'dropout': 0.2934514489816703, 'lr': 0.001179732267573371, 'weight_decay': 4.924644094938498e-05}. Best is trial 1 with value: 0.8680975307291097.
[I 2026-01-22 11:13:17,770] Trial 2 finished with value: 0.2773954116059379 and parameters: {'num_layers': 4, 'dropout': 0.1340953522563698, 'lr': 0.0004330377685736948, 'weight_decay': 1.9462223689985306e-06}. Best is trial 1 with value: 0.8680975307291097.
[I 2026-01-22 11:13:17,941] Trial 3 finished with value: 0.5531619278080258 and parameters: {'num_layers': 3, 'dropout': 0.5673622775924636, 'lr': 0.00016431367163934142, 'weight_decay': 8.626772341502056e-05}. Best is trial 1

Best hyperparameters: {'num_layers': 5, 'dropout': 0.2934514489816703, 'lr': 0.001179732267573371, 'weight_decay': 4.924644094938498e-05}

Running final training GIN...
{'num_layers': 5, 'dropout': 0.2934514489816703, 'lr': 0.001179732267573371, 'weight_decay': 4.924644094938498e-05}
Epoch 001 | Loss=1.1386 | TestAcc=0.526 | F1=0.363 | AUC=0.306 | Time=0.05s
Epoch 002 | Loss=0.6492 | TestAcc=0.526 | F1=0.363 | AUC=0.342 | Time=0.10s
Epoch 003 | Loss=0.4360 | TestAcc=0.474 | F1=0.305 | AUC=0.100 | Time=0.15s
Epoch 004 | Loss=0.4529 | TestAcc=0.474 | F1=0.305 | AUC=0.131 | Time=0.20s
Epoch 005 | Loss=0.3224 | TestAcc=0.474 | F1=0.305 | AUC=0.083 | Time=0.25s
Epoch 006 | Loss=0.3108 | TestAcc=0.474 | F1=0.305 | AUC=0.081 | Time=0.30s
Epoch 007 | Loss=0.4187 | TestAcc=0.474 | F1=0.305 | AUC=0.100 | Time=0.35s
Epoch 008 | Loss=0.2647 | TestAcc=0.474 | F1=0.305 | AUC=0.408 | Time=0.40s
Epoch 009 | Loss=0.2882 | TestAcc=0.474 | F1=0.305 | AUC=0.447 | Time=0.46s
Epoch 010 | Loss=0.2338 | TestA

In [13]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_graph2vec_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          w_acc=0.5, w_f1=0.3, w_auc=0.2,
          embedding_dim=128,
          epochs=200,
          test_size=0.2,
          use_optuna=True,
          n_trials=10,
      )

Loaded dataset IMDB-MULTI for Graph2Vec: 1500 graphs, 3 classes


[I 2026-01-22 11:13:31,082] A new study created in memory with name: no-name-d4d30493-2121-464b-b5f8-2d2bec0a83c5


Running Optuna for Graph2Vec+SVM hyperparameter tuning...


[I 2026-01-22 11:13:40,224] Trial 0 finished with value: 0.4660962631118881 and parameters: {'C': 3.7081609439242307, 'gamma': 0.794867896034824}. Best is trial 0 with value: 0.4660962631118881.
[I 2026-01-22 11:13:48,847] Trial 1 finished with value: 0.4511775212712713 and parameters: {'C': 7.04920217670471, 'gamma': 4.7734681587402745}. Best is trial 0 with value: 0.4660962631118881.
[I 2026-01-22 11:13:57,497] Trial 2 finished with value: 0.4822643395800988 and parameters: {'C': 13.502181952579566, 'gamma': 0.003697059566281911}. Best is trial 2 with value: 0.4822643395800988.
[I 2026-01-22 11:14:06,197] Trial 3 finished with value: 0.4376106181362238 and parameters: {'C': 0.031197952076748407, 'gamma': 0.09059631512150243}. Best is trial 2 with value: 0.4822643395800988.
[I 2026-01-22 11:14:14,997] Trial 4 finished with value: 0.5106928852653914 and parameters: {'C': 0.26920531358498506, 'gamma': 0.015584615031485491}. Best is trial 4 with value: 0.5106928852653914.
[I 2026-01-22 1

Best hyperparameters (Graph2Vec+SVM): {'C': 0.26920531358498506, 'gamma': 0.015584615031485491}
Running final Graph2Vec embedding on train+test graphs...
Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on IMDB-MULTI -> Acc: 0.490, F1: 0.470, AUC: 0.668, Score: 0.520
Embedding time: 10.62s | SVM training time: 0.67s | Optuna time: 87.31s | Memory usage: 1480.48 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv
Loaded dataset IMDB-MULTI for Graph2Vec: 1500 graphs, 3 classes


...

Best hyperparameters (Graph2Vec+SVM): {'C': 25.682886783000537, 'gamma': 2.4017344636119597}
Running final Graph2Vec embedding on train+test graphs...


[I 2026-01-22 11:21:33,493] A new study created in memory with name: no-name-7ec3d2df-d1be-44e3-983d-3e99dd858ef4


Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on MUTAG -> Acc: 0.789, F1: 0.789, AUC: 0.855, Score: 0.803
Embedding time: 0.86s | SVM training time: 0.01s | Optuna time: 7.78s | Memory usage: 1492.80 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv
Loaded dataset MUTAG for Graph2Vec: 188 graphs, 2 classes
Running Optuna for Graph2Vec+SVM hyperparameter tuning...


[I 2026-01-22 11:21:34,196] Trial 0 finished with value: 0.8721515151515152 and parameters: {'C': 58.78223212855365, 'gamma': 0.0907127870714943}. Best is trial 0 with value: 0.8721515151515152.
[I 2026-01-22 11:21:34,918] Trial 1 finished with value: 0.6743333333333333 and parameters: {'C': 22.330691439991902, 'gamma': 0.004020741990001376}. Best is trial 0 with value: 0.8721515151515152.
[I 2026-01-22 11:21:35,624] Trial 2 finished with value: 0.8721515151515152 and parameters: {'C': 97.55333163674517, 'gamma': 0.059473586967023}. Best is trial 0 with value: 0.8721515151515152.
[I 2026-01-22 11:21:36,321] Trial 3 finished with value: 0.8451111111111111 and parameters: {'C': 24.789645785580827, 'gamma': 0.015011709325307066}. Best is trial 0 with value: 0.8721515151515152.
[I 2026-01-22 11:21:37,253] Trial 4 finished with value: 0.8701515151515152 and parameters: {'C': 16.104547148202524, 'gamma': 0.1271690869888018}. Best is trial 0 with value: 0.8721515151515152.
[I 2026-01-22 11:21

Best hyperparameters (Graph2Vec+SVM): {'C': 58.78223212855365, 'gamma': 0.0907127870714943}
Running final Graph2Vec embedding on train+test graphs...
Training final SVM on Graph2Vec embeddings...
Graph2Vec Results on MUTAG -> Acc: 0.842, F1: 0.839, AUC: 0.926, Score: 0.858
Embedding time: 0.85s | SVM training time: 0.01s | Optuna time: 8.21s | Memory usage: 1492.80 MB
Graph2Vec summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/g2v.csv


In [14]:
for dataset_name in ["IMDB-MULTI", "ENZYMES", "MUTAG"]:
  for seed in [42, 43, 44]:
      run_netlsd_pipeline(
          dataset_name=dataset_name,
          seed=seed,
          w_acc=0.5, w_f1=0.3, w_auc=0.2,
          test_size=0.2,
          use_optuna=True,
          n_trials=10,
      )

Loaded dataset IMDB-MULTI for NetLSD: 1500 graphs, 3 classes


[I 2026-01-22 11:21:43,083] A new study created in memory with name: no-name-45df0d62-72fe-4456-8c04-7d06b022d65b


Running Optuna for NetLSD+SVM hyperparameter tuning...


[I 2026-01-22 11:21:47,247] Trial 0 finished with value: 0.5374135279619351 and parameters: {'C': 81.96916636320651, 'gamma': 2.4651663073193326}. Best is trial 0 with value: 0.5374135279619351.
[I 2026-01-22 11:21:52,255] Trial 1 finished with value: 0.4608146234804839 and parameters: {'C': 0.0746529154705982, 'gamma': 0.0008905262824697937}. Best is trial 0 with value: 0.5374135279619351.
[I 2026-01-22 11:21:56,654] Trial 2 finished with value: 0.40970297107127385 and parameters: {'C': 0.4164293674594436, 'gamma': 0.22513247021379945}. Best is trial 0 with value: 0.5374135279619351.
[I 2026-01-22 11:22:00,772] Trial 3 finished with value: 0.5237962046731358 and parameters: {'C': 26.965277702042147, 'gamma': 0.12546595180590156}. Best is trial 0 with value: 0.5374135279619351.
[I 2026-01-22 11:22:06,010] Trial 4 finished with value: 0.5050250408031368 and parameters: {'C': 64.85674419033406, 'gamma': 0.009910377943724668}. Best is trial 0 with value: 0.5374135279619351.
[I 2026-01-22 

Best hyperparameters (NetLSD+SVM): {'C': 81.96916636320651, 'gamma': 2.4651663073193326}
Running final NetLSD embedding on train+test graphs...
Training final SVM on NetLSD embeddings...
NetLSD Results on IMDB-MULTI -> Acc: 0.473, F1: 0.461, AUC: 0.624, Score: 0.500
Embedding time: 5.28s | SVM training time: 1.23s | Optuna time: 45.13s | Memory usage: 1496.86 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
Loaded dataset IMDB-MULTI for NetLSD: 1500 graphs, 3 classes


...

Best hyperparameters (NetLSD+SVM): {'C': 67.19009815548374, 'gamma': 5.843143340246375}
Running final NetLSD embedding on train+test graphs...


[I 2026-01-22 11:25:32,809] A new study created in memory with name: no-name-c1a5c52e-7d15-4708-ab37-f8a6f3ad1f97


Training final SVM on NetLSD embeddings...
NetLSD Results on MUTAG -> Acc: 0.842, F1: 0.834, AUC: 0.868, Score: 0.845
Embedding time: 0.52s | SVM training time: 0.01s | Optuna time: 4.29s | Memory usage: 1527.30 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
Loaded dataset MUTAG for NetLSD: 188 graphs, 2 classes
Running Optuna for NetLSD+SVM hyperparameter tuning...


[I 2026-01-22 11:25:33,258] Trial 0 finished with value: 0.6793333333333333 and parameters: {'C': 0.8880727709322209, 'gamma': 0.012999337729915883}. Best is trial 0 with value: 0.6793333333333333.
[I 2026-01-22 11:25:33,683] Trial 1 finished with value: 0.6783333333333333 and parameters: {'C': 0.4629197614768261, 'gamma': 0.0003096684486221798}. Best is trial 0 with value: 0.6793333333333333.
[I 2026-01-22 11:25:34,119] Trial 2 finished with value: 0.6793333333333333 and parameters: {'C': 13.88582700019075, 'gamma': 0.0001154011292969847}. Best is trial 0 with value: 0.6793333333333333.
[I 2026-01-22 11:25:34,542] Trial 3 finished with value: 0.897576379974326 and parameters: {'C': 1.2909118671767277, 'gamma': 4.921756179466289}. Best is trial 3 with value: 0.897576379974326.
[I 2026-01-22 11:25:35,177] Trial 4 finished with value: 0.8222272727272728 and parameters: {'C': 0.5501418134388858, 'gamma': 1.0464965915085642}. Best is trial 3 with value: 0.897576379974326.
[I 2026-01-22 11:

Best hyperparameters (NetLSD+SVM): {'C': 29.56904956730278, 'gamma': 9.70897221600041}
Running final NetLSD embedding on train+test graphs...
Training final SVM on NetLSD embeddings...
NetLSD Results on MUTAG -> Acc: 0.868, F1: 0.867, AUC: 0.914, Score: 0.877
Embedding time: 0.52s | SVM training time: 0.01s | Optuna time: 5.52s | Memory usage: 1527.30 MB
NetLSD summary stored in: /content/drive/MyDrive/InformationSystems/Classification/results/classification/netlsd.csv
